In [1]:
import numpy as np
import pickle
import os
# print(os.getcwd())

from eresh.evo import disloc_profile
from importlib.resources import files

default_specie = 'TiVZrB2' #TODO specify the specie to get the default parameters
sav_path = f'eresh/demo/'
if not os.path.exists(sav_path):
    os.makedirs(sav_path)

#* lattice grid parameters
int_stress_x, int_stress_y = 128, 128 #* grid size for the internal stress
fine_grid_int_stress = 2 #* grid per atomic spacing (0.5*b if set to 2)

disloc_type = 'edge' #* dislocation type, currently only 'edge' is supported
pn_pot_x, pn_pot_y = 513, 1 #* grid size for Peierls potential derivation, grid spacing is atomic spacing in default

evo_x, evo_y = 128, 128 #* grid size for dislocation evolution
fine_grid_evo = 2 #* grid per atomic spacing (0.5*b if set to 2)

#* empirical parameters
neb_image = 51 #* number of images for NEB calculation for PN barrier derivation
outercut = 750 #* outer cut-off radius for the dislocation profile, angstrom
external_load_list = np.linspace(0,100,1001)/160.2176621 #* external load list, eV/A^3

disloc = \
    disloc_profile(
        #* basic structural parameters
        specie_denote = default_specie, 
        lattice_info = np.load(str(files('eresh.demo.lattice_info')/f'{default_specie}.npz')), 
        stiffness_tensor_buffer = np.load(str(files('eresh.demo.cij')/f'{default_specie}.npz')),
        constituent_info = np.load(str(files('eresh.demo.composition')/f'{default_specie}.npz')), #* use default_specie
        
        gsfe_func = None, #* use default parameterization of generalized stacking fault energy
        gsfe_coef_buffer = np.load(str(files('eresh.demo.gsfe')/f'{default_specie}.npz')), #* use default_specie
        
        #! under construction
        use_internal_stress = True, #* use internal stress as quenched noise during evolution
        int_stress_grid = [int_stress_x, int_stress_y], #* grid per atomic spacing for internal stress
        fine_grid_int_stress = fine_grid_int_stress, #* grid per atomic spacing for internal stress
        internal_stress_buffer = np.load(str(files('eresh.demo.int_stress')/f'{default_specie}/{int_stress_x}_{int_stress_y}_f{fine_grid_int_stress}_{fine_grid_int_stress}.npz')),
        
        disloc_type = disloc_type,
        outer_cut = outercut,
        # lt_generator = pickle.load(open(str(files('eresh.demo.lineE_generator')/f'{default_specie}.pickle'), 'rb')),
        lt_generator = None,
        
        pn_calc = True, #* calculate local Peierls potential
        pn_pot_grid = [pn_pot_x, pn_pot_y], #* grid size for Peierls potential derivation
        pn_kernel_buffer = pickle.load(open(str(files('eresh.demo.kernel')/f'{default_specie}_pfdd.pkl'), 'rb')),
        pn_weight_buffer = np.load(str(files('eresh.demo.int_stress')/f'{default_specie}/pnweight_{int_stress_x}_{int_stress_y}_f{fine_grid_int_stress}_{fine_grid_int_stress}_smearing.npz')),
        pn_savpth = sav_path,
        
        neb_image = neb_image,
        
        external_load_list = external_load_list, #* external load list for dislocation evolution
        
        evo_grid = [evo_x, evo_y], #* grid size for dislocation evolution
        fine_grid_evo = fine_grid_evo, #* grid per atomic spacing for dislocation evolution
        
        evo_sav_path = sav_path, #* save path for the dislocation profile and evolution results
        threads_num = 12, #* number of threads to use
        verbose = False, #* print essential information during calculation
    )

In [2]:
disloc.evolute()

Stacking fault energy: 0.026793175589619397 eV/A^2
Start line energy prefactor calculation for TiVZrB2 edge dislocation
Line energy prefactor: 2.732784656333261 eV/A
Loading internal stress field by default
Start local PN potential parameterization
NEB runs completed for ['ZrB2', 'VB2', 'TiB2']
Start evolution
tau = 36.3 GPa; Max iterations reached: 150000,                     grad norm = 0.6078982265020056 0.6482199979313314                     k_fitavepos = 0.0041936681237920515
Critical stress reached, information saved to eresh/demo//evo_runs/TiVZrB2edge.pickle


Basic idea is to utilize shear stress field due to lattice misfit; localized Peierls stress; line tension into the evolution of dislocation profile.

Parameters brief intro.:
- `specie_denote`: Denotion of diboride specie, e.g. `"50Ti50ZrB2"`.
- `lattice_info`: Lattice parameter of hexagonal lattice along a-/c-axis, respectively. Lattice vector of the stiffness tensor for the high entropy diborides. (e.g. `{'lattice_param': [3.172, 3.546], 'lattice_vec': np.diag([3.172, 3.546, 5.494])}` for ZrB2 in [1120]-[0001]-[1100] coordinate system). Demonstration is in `demo/lattice_param` folder. Unit in Angstrom.
- `stiffness_tensor_buffer`: Stiffness tensor in Voigt notation, the buffer should contain Demonstration is in `demo/cij` folder. Stiffness tensor will be utilized in [1123]-[1123]-[1100] coordinate system during calculation. Unit in GPa.
- `constituent_info`: Constituent binary diborides and corresponding concentration. The format is a dictionary with keys as binary diborides and values as their concentrations (e.g., `{'TiB2': 0.5, 'ZrB2': 0.5}` for (50Ti-50Zr)B2). Sum of concentrations should be 1.
- `gsfe_func`: Parameterization of generalized stacking fault energy and corresponding shear stress. If `gsfe_func=None`, use the default parametrization method and corresponding Fourier coefficients by calling `utils.ft_main_vec`.
- `gsfe_coef_buffer`: If use default parameterization method, the gsfe' and gradient of gsfe's parameterization coefficients of the default specie will be loaded.
- `use_internal_stress`: If `True`, the internal stress will be utilized into evolution of dislocation profile.
- `int_stress_grid`: Dimensionality of internal stress map, e.g. `[128, 128]`.
- `fine_grid_int_stress`: Grid spacing of internal stress map, e.g. `... = 2` means grid spacing is `b/2` where `b` is the magnitude of Burgers vector.
- `internal_stress_buffer`: Internal stress as quenched noise based on micromechanics and lattice eigenstrain theory ([Geslin et al](https://doi.org/10.1016/j.jmps.2021.104480)). Currently the derivation program is not available but we provide some of the calculated internal stress maps in `demo/internal_stress` folder and each map for particular composition contains $\sigma_{13}$ and $\sigma_{23}$'s information.
- `disloc_type`: Type of dislocation, currently only support edge dislocaition, e.g. `disloc_type = "edge"`.
- `outer_cut`: Outer cut-off radius for path integral of stress. (e.g. experimental dislocation spacing)
- `lt_generator`: Line energy prefactor generator, if `lt_generator=None`, derive prefactor from scratch (e.g. $K_{\theta}+K^{''}_{\theta}$)
- `pn_calc`: Turn on/off PN pot calculation.
- `pn_pot_grid`: Lattice grid dim for PN potential calculation. 1D grid is sufficient.
- `pn_kernel_buffer`: Since PFDD is utilized to calculate the PN potential, the pre-calculated kernel should either be provided or derived from scratch. If `pn_kernel_buffer=None`, derive kernel from scratch.
- `pn_weight_buffer`: Interpolation of local PN pot to simulate the high-entropy atomic environment. This term is under progress and currently some calculated weights are provided.
- `pn_savpth`: Save path of PN potential.
- `neb_image`: Number of images for NEB calculation for derive the Peierls barrier.
- `external_load_list`: List of external shear stress to be applied during NEB calculation. Following increasing order.
- `evo_grid`: Grid dimension for dislocation profile evolution. (Main function)
- `fine_grid_evo`: Grid spacing for dislocation profile evolution. e.g. `... = 2` means grid spacing is `b/2` where `b` is the magnitude of Burgers vector. (Main function)
- `evo_sav_path`: Save path of dislocation profile evolution. (Main function)
- `threads_num`: Number of threads to be used during whole calculation.
- `verbose`: Verbosity level during calculation.

Ref.:
- [Koslowski et al](https://doi.org/10.1016/S0022-5096\(02\)00037-6) J. Mech. Phys. Solids 50 (2002) 2597.
- [Geslin et al](https://doi.org/10.1016/j.jmps.2021.104480) J. Mech. Phys. Solids 153 (2021) 104480.
- [Vaid et al](https://doi.org/10.1016/j.actamat.2022.118095) Acta Mater. 236 (2022) 118095.
- [Han et al](https://doi.org/10.1016/j.actamat.2025.121367) Acta Mater. 297 (2025) 121367.